In [1]:
using Distributions
using StatsBase
using DataFrames
using Flux;
using DataStructures;

In [6]:
n_disks,n_pegs=3,3;

In [11]:
function checkLegal(State,act) #Check the legality of the move: Moving the top disc from rung m to rung n when state=State
    if State[act[1]]==[]
        return 0
    elseif State[act[2]]==[]
        return 1
    elseif State[act[1]][1]>State[act[2]][1]
        return 0
    else 
        return 1
    end
end

function move(State,act)
    newState=copy(State)
    if checkLegal(State,act)>0
        newState[act[2]]=[newState[act[1]][1];newState[act[2]]]
        newState[act[1]]=newState[act[1]][2:end]
        return newState
    else
        newState=State;
        #println("Illegal Move")
        return newState
    end
    
end

move (generic function with 1 method)

In [12]:
State=[[1,2,3],[],[]]; #State is specified with the number indicating the size of the disc and represent the discs arranged top to bottom

finalState=[[],[1,2,3],[]]

all_moves=vec([[i,j] for i=1:3,j=1:3 if !(i==j)])

neighbour(a)=[move(a,s) for s in all_moves if move(a,s)!=a]

function all_legal(State)
    legal_moves=[];
    for move in all_moves
        if checkLegal(State,move)>0
            push!(legal_moves,move)
        end
    end
    return legal_moves;
end

function depthFirstTraversal(start_node)
    visited = [];
    stack = [start_node];

    while !isempty(stack)
        node = pop!(stack)
        if !(node in visited)
            println(node)
            push!(visited, node)
            for neighbor in neighbour(node)
                push!(stack, neighbor)
            end
        end
    end
    
    return visited
end

all_states=depthFirstTraversal(State);

move_to_ix=Dict();
ix_to_move=Dict();
for (ix,move) in enumerate(all_moves)
    move_to_ix[move]=ix
    ix_to_move[ix]=move
end

state_to_ix=Dict();
ix_to_state=Dict();
for (ix,state) in enumerate(all_states)
    state_to_ix[state]=ix
    ix_to_state[ix]=state
end

Vector{Any}[[1, 2, 3], [], []]
Vector{Any}[[2, 3], [], [1]]
Vector{Any}[[2, 3], [1], []]
Vector{Any}[[3], [1], [2]]
Vector{Any}[[3], [], [1, 2]]
Vector{Any}[[], [3], [1, 2]]
Vector{Any}[[], [1, 3], [2]]
Vector{Any}[[2], [1, 3], []]
Vector{Any}[[2], [3], [1]]
Vector{Any}[[], [2, 3], [1]]
Vector{Any}[[], [1, 2, 3], []]
Vector{Any}[[1], [2, 3], []]
Vector{Any}[[1], [3], [2]]
Vector{Any}[[1, 2], [3], []]
Vector{Any}[[1, 2], [], [3]]
Vector{Any}[[2], [], [1, 3]]
Vector{Any}[[2], [1], [3]]
Vector{Any}[[], [1], [2, 3]]
Vector{Any}[[], [], [1, 2, 3]]
Vector{Any}[[1], [], [2, 3]]
Vector{Any}[[1], [2], [3]]
Vector{Any}[[], [2], [1, 3]]
Vector{Any}[[], [1, 2], [3]]
Vector{Any}[[3], [1, 2], []]
Vector{Any}[[3], [2], [1]]
Vector{Any}[[1, 3], [2], []]
Vector{Any}[[1, 3], [], [2]]


In [ ]:
function StackedMatrix(State) #Visualising the game-state as a stacked matrix
    StateMat=copy(State)
    n=n_disks;
    m=n_pegs;
    for j in 1:m
        tempvec=ones(n)
        if length(StateMat[j])<n
            for i in 1:n-length(StateMat[j])
                tempvec[i]=0
            end
            if length(StateMat[j])!=0
                for i in 1:length(StateMat[j])
                    tempvec[i+n-length(StateMat[j])]=StateMat[j][i]
                end
            end
        end
        StateMat[j]=tempvec
    end
    return StateMat
end

In [19]:
function FloydWarshall(dist)
    println("Initial Adjacency matrix: ")
    display(dist)
    println("\n")
    n=length(all_states)
    for k in 1:n
        println("Matrix A^k with k as ",k)
        println("Current dist:")
        display(dist)
        println("\n")
        for i in 1:n
            for j in 1:n 
                dist[i,j] = min(dist[i,j], dist[i,k] + dist[k,j])
            end
        end

    end
    println("\n Matrix after applying Floyd Warshall: ")
    display(dist)
    println("\n")
    return dist
end

M=fill(Inf,length(all_states),length(all_states))
for i in 1:length(all_states)
    for j in 1:length(all_states)
        M[i,j]= all_states[j] in neighbour(all_states[i]) ? 1 : Inf
    end
end

minDistMat=FloydWarshall(M)


function minMoves(State1,State2,M)
    i=findall(x->x==State1,all_states)[1]
    j=findall(x->x==State2,all_states)[1]
    return M[i,j]
end

Initial Adjacency matrix: 


27×27 Matrix{Float64}:
 Inf    1.0   1.0  Inf   Inf   Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
  1.0  Inf    1.0  Inf   Inf   Inf      Inf   Inf   Inf    1.0  Inf   Inf
  1.0   1.0  Inf    1.0  Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf    1.0  Inf    1.0  Inf      Inf   Inf   Inf   Inf   Inf    1.0
 Inf   Inf   Inf    1.0  Inf    1.0     Inf   Inf   Inf   Inf   Inf    1.0
 Inf   Inf   Inf   Inf    1.0  Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf    1.0     Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf    1.0     Inf   Inf   Inf   Inf   Inf   Inf
  ⋮          



Matrix A^k with k as 1
Current dist:


27×27 Matrix{Float64}:
 Inf    1.0   1.0  Inf   Inf   Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
  1.0  Inf    1.0  Inf   Inf   Inf      Inf   Inf   Inf    1.0  Inf   Inf
  1.0   1.0  Inf    1.0  Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf    1.0  Inf    1.0  Inf      Inf   Inf   Inf   Inf   Inf    1.0
 Inf   Inf   Inf    1.0  Inf    1.0     Inf   Inf   Inf   Inf   Inf    1.0
 Inf   Inf   Inf   Inf    1.0  Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf    1.0     Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf   …  Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf   Inf      Inf   Inf   Inf   Inf   Inf   Inf
 Inf   Inf   Inf   Inf   Inf    1.0     Inf   Inf   Inf   Inf   Inf   Inf
  ⋮          



Matrix A^k with k as 2
Current dist:


...



Matrix A^k with k as 27
Current dist:


27×27 Matrix{Float64}:
 2.0  1.0  1.0  2.0  3.0  4.0  5.0  6.0  …  5.0  5.0  4.0  3.0  2.0  3.0  3.0
 1.0  2.0  1.0  2.0  3.0  4.0  5.0  6.0     4.0  4.0  3.0  2.0  1.0  2.0  3.0
 1.0  1.0  2.0  1.0  2.0  3.0  4.0  5.0     5.0  5.0  4.0  3.0  2.0  3.0  2.0
 2.0  2.0  1.0  2.0  1.0  2.0  3.0  4.0     6.0  6.0  5.0  4.0  3.0  4.0  1.0
 3.0  3.0  2.0  1.0  2.0  1.0  2.0  3.0     7.0  7.0  6.0  5.0  4.0  5.0  1.0
 4.0  4.0  3.0  2.0  1.0  2.0  1.0  2.0  …  7.0  6.0  7.0  6.0  5.0  6.0  2.0
 5.0  5.0  4.0  3.0  2.0  1.0  2.0  1.0     6.0  5.0  6.0  7.0  6.0  7.0  3.0
 6.0  6.0  5.0  4.0  3.0  2.0  1.0  2.0     5.0  4.0  5.0  6.0  7.0  7.0  4.0
 7.0  7.0  6.0  5.0  4.0  3.0  2.0  1.0     5.0  4.0  5.0  6.0  7.0  7.0  5.0
 7.0  7.0  6.0  5.0  4.0  3.0  3.0  2.0     6.0  5.0  6.0  7.0  8.0  8.0  5.0
 7.0  7.0  6.0  5.0  4.0  3.0  3.0  3.0  …  7.0  6.0  7.0  8.0  8.0  9.0  5.0
 6.0  6.0  5.0  4.0  3.0  2.0  2.0  3.0     7.0  6.0  7.0  8.0  7.0  8.0  4.0
 5.0  5.0  4.0  3.0  2.0  1.0  1.0  2.0  




 Matrix after applying Floyd Warshall: 


27×27 Matrix{Float64}:
 2.0  1.0  1.0  2.0  3.0  4.0  5.0  6.0  …  5.0  5.0  4.0  3.0  2.0  3.0  3.0
 1.0  2.0  1.0  2.0  3.0  4.0  5.0  6.0     4.0  4.0  3.0  2.0  1.0  2.0  3.0
 1.0  1.0  2.0  1.0  2.0  3.0  4.0  5.0     5.0  5.0  4.0  3.0  2.0  3.0  2.0
 2.0  2.0  1.0  2.0  1.0  2.0  3.0  4.0     5.0  5.0  4.0  3.0  3.0  2.0  1.0
 3.0  3.0  2.0  1.0  2.0  1.0  2.0  3.0     5.0  5.0  4.0  3.0  3.0  2.0  1.0
 4.0  4.0  3.0  2.0  1.0  2.0  1.0  2.0  …  6.0  6.0  5.0  4.0  4.0  3.0  2.0
 5.0  5.0  4.0  3.0  2.0  1.0  2.0  1.0     6.0  5.0  6.0  5.0  5.0  4.0  3.0
 6.0  6.0  5.0  4.0  3.0  2.0  1.0  2.0     5.0  4.0  5.0  6.0  6.0  5.0  4.0
 7.0  7.0  6.0  5.0  4.0  3.0  2.0  1.0     5.0  4.0  5.0  6.0  7.0  6.0  5.0
 7.0  7.0  6.0  5.0  4.0  3.0  3.0  2.0     6.0  5.0  6.0  7.0  7.0  6.0  5.0
 7.0  7.0  6.0  5.0  4.0  3.0  3.0  3.0  …  7.0  6.0  7.0  7.0  7.0  6.0  5.0
 6.0  6.0  5.0  4.0  3.0  2.0  2.0  3.0     7.0  6.0  7.0  6.0  6.0  5.0  4.0
 5.0  5.0  4.0  3.0  2.0  1.0  1.0  2.0  

minMoves (generic function with 1 method)

In [33]:

function dijkstra_with_path(graph, start)
    n = length(graph)
    visited = falses(n)
    dist = fill(Inf, n)
    dist[start] = 0
    prev = fill(0, n)

    pq = PriorityQueue()
    enqueue!(pq, start, 0)

    while !isempty(pq)
        current = dequeue!(pq)

        if visited[current]
            continue
        end

        visited[current] = true

        for (neighbor, weight) in graph[current]
            if !visited[neighbor]
                alt = dist[current] + weight
                if alt < dist[neighbor]
                    dist[neighbor] = alt
                    prev[neighbor] = current
                    enqueue!(pq, neighbor, alt)
                end
            end
        end
    end

    return dist, prev
end

function reconstruct_path(prev, start, target)
    path = []
    current = target
    while current != start
        push!(path, current)
        current = prev[current]
        if current == 0
            return "No path"
        end
    end
    push!(path, start)
    return reverse(path)
end

# Example graph
graph=Dict(state_to_ix[init_state]=>[(state_to_ix[to_state],1) for to_state in neighbour(init_state)] for init_state in all_states)

start_node = state_to_ix[State];
end_node = state_to_ix[finalState];

distances, predecessors = dijkstra_with_path(graph, start_node)
path = reconstruct_path(predecessors, start_node, end_node)

println("Shortest path from $start_node to $end_node: ", path)


Shortest path from 1 to 11: Any[1, 3, 4, 5, 6, 13, 12, 11]


In [35]:
[ix_to_state[t] for t in path]

8-element Vector{Vector{Vector{Any}}}:
 [[1, 2, 3], [], []]
 [[2, 3], [1], []]
 [[3], [1], [2]]
 [[3], [], [1, 2]]
 [[], [3], [1, 2]]
 [[1], [3], [2]]
 [[1], [2, 3], []]
 [[], [1, 2, 3], []]

In [36]:
path

8-element Vector{Any}:
  1
  3
  4
  5
  6
 13
 12
 11